# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff63ffdc280>
├── 'a' --> tensor([[ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018]])
└── 'x' --> <FastTreeValue 0x7ff66c9061f0>
    └── 'c' --> tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                        [-1.1731, -0.1371, -0.5604, -0.9793],
                        [ 1.2938,  0.1326,  1.0591, -0.7810]])

In [4]:
t.a

tensor([[ 0.1310, -0.4369,  1.1909],
        [-0.9809,  0.1708, -0.9018]])

In [5]:
%timeit t.a

74.5 ns ± 0.173 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff63ffdc280>
├── 'a' --> tensor([[-0.9938,  0.0575,  1.5686],
│                   [ 0.7634,  0.0952, -0.6779]])
└── 'x' --> <FastTreeValue 0x7ff66c9061f0>
    └── 'c' --> tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                        [-1.1731, -0.1371, -0.5604, -0.9793],
                        [ 1.2938,  0.1326,  1.0591, -0.7810]])

In [7]:
%timeit t.a = new_value

79.1 ns ± 0.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018]]),
    x: Batch(
           c: tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                      [-1.1731, -0.1371, -0.5604, -0.9793],
                      [ 1.2938,  0.1326,  1.0591, -0.7810]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1310, -0.4369,  1.1909],
        [-0.9809,  0.1708, -0.9018]])

In [11]:
%timeit b.a

68.9 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.7970, -0.3021, -0.1870],
               [ 0.7489, -0.5244, -0.6330]]),
    x: Batch(
           c: tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                      [-1.1731, -0.1371, -0.5604, -0.9793],
                      [ 1.2938,  0.1326,  1.0591, -0.7810]]),
       ),
)

In [13]:
%timeit b.a = new_value

580 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

974 ns ± 4.58 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 679 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff5b7b96f40>
├── 'a' --> tensor([[[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]],
│           
│                   [[ 0.1310, -0.4369,  1.1909],
│                    [-0.9809,  0.1708, -0.9018]]])
└── 'x' --> <FastTreeValue 0x7ff6441c1e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff5b7be6820>
├── 'a' --> tensor([[ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018],
│                   [ 0.1310, -0.4369,  1.1909],
│                   [-0.9809,  0.1708, -0.9018]])
└── 'x' --> <FastTreeValue 0x7ff5c926d6d0>
    └── 'c' --> tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                        [-1.1731, -0.1371, -0.5604, -0.9793],
                 

In [23]:
%timeit t_cat(trees)

37.9 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.7 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]],
       
               [[ 0.1310, -0.4369,  1.1909],
                [-0.9809,  0.1708, -0.9018]]]),
    x: Batch(
           c: tensor([[[-0.2818,  0.3983,  0.9330,  0.6824],
                       [-1.1731, -0.1371, -0.5604, -0.9793],
                       [ 1.2938,  0.1326,  1.0591, -0.7810]],
         

In [26]:
%timeit Batch.stack(batches)

89.5 µs ± 698 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018],
               [ 0.1310, -0.4369,  1.1909],
               [-0.9809,  0.1708, -0.9018]]),
    x: Batch(
           c: tensor([[-0.2818,  0.3983,  0.9330,  0.6824],
                      [-1.1731, -0.1371, -0.5604, -0.9793],
                      [ 1.2938,  0.1326,  1.0591, -0.7810],
                      [-0.2818,  0.3983,  0.9330,  0.6824],
                      [-1.1731, -

In [28]:
%timeit Batch.cat(batches)

168 µs ± 614 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

385 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
